# Desafio IA SERPRO 2020 -  PLN

Robson de Sousa Martins
________________________________________________________________________________________________________________________

**Página no Desafio:** http://evalai.dev.serpro/web/challenges/challenge-page/15/overview

## Bibliotecas Utilizadas

In [ ]:
import pandas as pd
import numpy as np
import csv
import string

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords as sw

import re
from unicodedata import normalize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV
from statistics import mean 

import spacy

In [ ]:
# Clssificadores
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB  
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import NearestCentroid
from sklearn.gaussian_process import GaussianProcessClassifier


## Download de Módulos

In [ ]:
!python -m spacy download pt
nltk.download('stopwords')
nltk.download('punkt')


     |████████████████████████████████| 21.2MB 1.2MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp36-none-any.whl size=21186282 sha256=5eae05eba4fe239b27da6c18131fe33ea9c6bff99341b0848bb905fadfa2a40e
  Stored in directory: /tmp/pip-ephem-wheel-cache-vx6s480l/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Funções

In [ ]:
# Determinando forma básica (lema) das palavras
# Isso só vale pra verbos ("colocar no infinitivo")
def lemmatizer(text):
  sent = []
  # uso o lemmatizer do spacy
  doc = nlp(text)
  for word in doc:
    if word.pos_ == "VERB":
      sent.append(word.lemma_)
    else:
      sent.append(word.orth_)
  return " ".join(sent)

In [ ]:
# Transforma números em palavras por extenso
# Isso acabou por ser relevante na base de dados trabalhada, e elevou a 
# performance dos modelos

# Marcos Paulo Ferreira (Daemonio)
# https://daemoniolabs.wordpress.com
# Versão 1.1 by daemonio @Sat Dec 20 23:41:50 BRST 2014
class dExtenso():
    trioextenso=()
    classextenso=()
 
    def __init__(self):
        self.trioextenso=(
                     ("dummy","um","dois","três","quatro","cinco","seis","sete",
                     "oito","nove"),
                     ("dez","onze","doze","treze","quatorze","quinze","dezesseis",
                     "dezessete","dezoito","dezenove"),
                     ("dummy","dummy","vinte","trinta","quarenta","cinquenta",
                     "sessenta","setenta","oitenta","noventa"),
                     ("dummy","cento","duzentos","trezentos","quatrocentos",
                     "quinhentos","seiscentos","setecentos","oitocentos",
                     "novecentos"))
        self.classextenso=(
                      "dummy","mil","milh","bilh","trilh","quatrilh",
                      "quintilh","sextilh","septilh","octilh",
                      "nonilh","decilh","undecilh","duodecilh",
                      "tredecilh","quatordecilh","quindecilh",
                      "sexdecilh","setedecilh","octodecilh",
                      "novedecilh","vigesilh" )
  
    def escrever_trio_extenso(self, trio):
        saida=[]
        if trio == '100':
            return 'cem'
        elif trio == '000':
            return 'zero'
        else:
            c, d, u = trio
            c, d, u = int(c), int(d), int(u)
            if c != 0:
                saida.append(self.trioextenso[3][c])
            if d == 1:
                saida.append(self.trioextenso[1][u])
            else:
                if d != 0:
                    saida.append(self.trioextenso[2][d])
                if u != 0:
                    saida.append(self.trioextenso[0][u])
        return ' e '.join(saida)
  
    def nao_e_ultimo_trio(self, totalTrios, contador):
        return contador < (totalTrios - 1)
  
    def trio_a_esquerda_eq_zero(self, trioLista, contador):
        t = len(trioLista)-1
        return trioLista[t-contador-1] == '000'
  
    def getExtenso(self, num, quebradelinhas=0):
        # by Robson Martins
        if num == "r$": # R$ xxxx,xx
          return "real"
        if num == "us$": # US$ xxxx,xx
          return "dólar"
        num = re.sub(r'[\.]{1}','',num) # x.xxx.xxx
        num = re.sub(r'[\,]{1}[0-9]+','',num) # xxx,yyy
        '''
        Ordinais: aqui estou transformando ordinais em números cardinais por
        extenso.
        TODO: uma possível melhora é converter ordinais para extenso
        '''
        num = re.sub(r'([0-9]+)[oaªº°]{1}',r'\1',num)
        if not num.isnumeric():
          return num
        # / by Robson Martins
        num = num.lstrip('0')
        pad = 3 - len(num)%3
        if pad < 3: num = '0'*pad + num
        it = iter(num)
        trioLista = [ ''.join([a,b,c]) for a, b, c in zip(it, it, it)]
        if len(trioLista) > len(self.classextenso):
            raise IndexError
        contador=0
        saida=''
        extensofinal=''
        for trio in reversed(trioLista):
            trioInt=int(trio)
            if trioInt > 0:
                saida = self.escrever_trio_extenso(trio)
                if contador > 0:
                    saida = saida + ' ' + self.classextenso[contador]
                if contador > 1:
                    if trioInt > 1:
                        saida = saida + 'ões'
                    else:
                        saida = saida + 'ão'
                if quebradelinhas == 0:
                    if self.nao_e_ultimo_trio(len(trioLista), contador):
                        if self.trio_a_esquerda_eq_zero(trioLista, contador):
                            saida = ' e ' + saida
                        elif trioInt >= 100:
                            saida = ', ' + saida
                        else:
                            saida = ' e ' + saida
                else:
                    saida = saida + '\n'
                extensofinal = saida + extensofinal
            contador = contador + 1
        return extensofinal.rstrip('\n')


In [ ]:
# Realiza uma limpeza básica de um texto, preparando-o para classificação.
def limpar_texto(texto):
    # Converte para minúsculas
    texto = texto.lower()
    # Troca números para extenso
    extenso = dExtenso()
    tokens = tokenizer.tokenize(texto)
    tokens = [extenso.getExtenso(palavra.strip()) for palavra in tokens]
    texto = ' '.join(tokens)  
    # Remove números, se restaram
    texto = re.sub(r'[0-9]+',' ',texto)
    # Remove pontuacao
    texto = texto.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    # Remove espacos extras
    texto = re.sub(r'\s+',' ',texto)
    # Remove stopwords
    tokens = tokenizer.tokenize(texto)
    tokens = [palavra.strip() for palavra in tokens if palavra not in stopwords]
    texto = ' '.join(tokens)  
    # Lematiza palavras
    texto = lemmatizer(texto)
    # Remove acentos
    texto = normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')
    # cria dict de palavras unicas
    # remove palavras menores que 2 caracteres
    tokens = tokenizer.tokenize(texto)
    fdist = nltk.FreqDist(tokens)
    tokens = [palavra.strip() for palavra, freq in fdist.items() if len(palavra) >= 2]
    texto = ' '.join(tokens)  
    return texto

In [ ]:
# Treina um classificador, otimiza hiperparâmetros,
# avalia performance e retorna métricas de desempenho
def build(X,y,vec,est,grid):
  est_name = est.__class__.__name__
  vec_name = vec.__class__.__name__
  print('Testando o classificador',est_name,'-',vec_name,'...')
  # Massa de Dados
  X_data = vec.transform(X.tolist()).toarray()
  y_data = y.tolist()
  # Otimiza modelos
  # Uso F1 Macro como métrica
  clf = GridSearchCV(est,grid,scoring='f1_macro',n_jobs=-1,cv=5,verbose=100)
  clf.fit(X_data,y_data)
  # Obtém as métricas de desempenho - o quanto nosso classificador acertou?
  return clf.best_estimator_, est_name, vec_name, clf.best_score_

## Inicialização

In [ ]:
# Tokenizador: utilizado para separar uma frase em palavras
tokenizer = RegexpTokenizer(r'\w+')

# stopwords do português
stopwords = nltk.corpus.stopwords.words('portuguese')

# lib spacy para fazer o lemmatizer
nlp = spacy.load('pt')

# Semente aleatória a ser usada ao longo desse notebook.
# Procure manter sempre a mesma semente aleatória. Desse modo, poderá comparar a evolução entre diferentes técnicas
random_state=660601

# Nome do arquivo fornecido pelo desafio com os dados rotulados para treino
nome_arquivo_com_rotulos_para_treino = 'treino.csv'

# Nome do arquivo fornecido pelo desafio com os dados não rotulados, que deverão ser classificados pelo modelo construído aqui
nome_arquivo_sem_rotulos = 'teste-sem-classe.csv'

# Nome do arquivo que será criado com os rótulos gerados pelo classificador
# Esse é o arquivo se será submetido à página do desafio
nome_arquivo_rotulado_classificador = 'teste.csv'

# Nome do arquivo com os dados rotulados para treino, após preprocessamento (gero isso pra verificar minha rotina de preprocessamento)
nome_arquivo_com_rotulos_para_treino_preprocessado = 'treino-preprocessado.csv'

# Nome do arquivo com os dados não rotulados, após preprocessamento (gero isso pra verificar minha rotina de preprocessamento)
nome_arquivo_sem_rotulos_preprocessado = 'teste-sem-classe-preprocessado.csv'

# Caminho do Google Drive onde os arquivos estão armazenados
# Uso a integração do Google Colab com o Google Drive
# Se for usar um Jupyter fora do Colab e os arquivos estiverem no mesmo nível do Notebook,
# coloque path_meu_google_drive = ''
path_meu_google_drive = '/content/drive/My Drive/Datasets/Desafio IA 2020 PLN/'

## Carregando os dados rotulados

Nessa etapa carregamos os dados rotulados fornecidos no desafio. Eles serão usados para treinamento de um classificador.

In [ ]:
path = path_meu_google_drive + nome_arquivo_com_rotulos_para_treino
df = pd.read_csv(path, index_col=None, engine='python', sep =',', encoding="utf-8")
print('Total de registros carregados:',len(df))

# Exibe uma amostra dos dados
df.head()

Total de registros carregados: 1700


,id,texto,classe
0,675,lula diz que senado tem maioridade para resolv...,neutro
1,1733,adolescente é morto por ouvir música alta nos ...,tristeza
2,1855,coreia do sul insinua que hackers ligados à co...,neutro
3,1144,mamãe foca dá selinho em filhote recém-nascido...,alegria
4,462,adolescente de 15 anos que estava sumida é ach...,tristeza


In [ ]:
# Distribuição das classes nos dados fornecidos
df.groupby('classe').count()

,id,texto
classe,,
alegria,156,156
desgosto,223,223
medo,189,189
neutro,461,461
raiva,70,70
surpresa,214,214
tristeza,387,387


## Preparando os textos para classificação

A preparação dos dados é uma das etapas mais importantes para se obter uma boa performance na classificação de textos, e pode significar a diferença entre o sucesso e o fracasso de um projeto.

In [ ]:
df['texto'] = df['texto'].apply(limpar_texto)
df.head()

,id,texto,classe
0,675,lula dizer senado maioridade resolver problema...,neutro
1,1733,adolescente morto ouvir musica alta estados un...,tristeza
2,1855,coreia sul insinuar hackers ligar norte atacar...,neutro
3,1144,mamae foca dar selinho filhote recem nascido b...,alegria
4,462,adolescente quinze anos sumir achar morta mana...,tristeza


In [ ]:
# Salva os registros de treino pre-processados
path = path_meu_google_drive + nome_arquivo_com_rotulos_para_treino_preprocessado
df.to_csv(path, index=False, encoding="utf-8", columns=['id','texto','classe'])

## Treinando e testando métodos de vetorização e classificador


In [ ]:
# Vetorizadores

countVectorizer = CountVectorizer()
countVectorizer.fit_transform(df['texto'].tolist())

tfidfVectorizer = TfidfVectorizer()
tfidfVectorizer.fit_transform(df['texto'].tolist())


<1700x8292 sparse matrix of type '<class 'numpy.float64'>'
	with 33660 stored elements in Compressed Sparse Row format>

```
# Combinações de classificador/vetorizador
# Vetorizador TfidfVectorizer
 
estimators = [
  {'est': MLPClassifier(), 
   'grid':{
      #'activation': ['identity', 'logistic', 'tanh', 'relu'],
      #'hidden_layer_sizes': [(50),(100),(200)],
      'random_state': [random_state],
      'max_iter':[200],
      #'solver': ['lbfgs', 'sgd', 'adam'],
      #'alpha': [1e-5,1e-4,1e-3,1e-2,1e-1,1.0],
      #'learning_rate': ['constant', 'invscaling', 'adaptive'],
      },
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0},
  {'est': MultinomialNB(), 
   'grid':{
#      'fit_prior' : {True},
#      'alpha': [1e-3,1e-2,1e-1,1.0],
#      'class_prior': [None],
      },
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0},
  {'est': LogisticRegression(), 
   'grid':{
      'random_state':[random_state],
      'max_iter':[10000],
      'n_jobs':[-1],
#      'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
#      'tol': [1e-4,1e-3,1e-2,1e-1,1.0],
#      'C': [1e-1,1.0,2.0,3.0],
#      'penalty': ['l1', 'l2', 'elasticnet', 'none'],
      },
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0},
  {'est': SGDClassifier(), 
   'grid':{
      'random_state':[random_state],
      'max_iter':[10000],
      'n_jobs':[-1],
#      'alpha': [1e-5],
#      'loss': ['hinge'],
#      'penalty': ['elasticnet'],
      },
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0},
  {'est': ExtraTreeClassifier(), 
   'grid':{'random_state':[random_state]},
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0},
  {'est': DecisionTreeClassifier(), 
   'grid':{'random_state':[random_state]},
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0},
  {'est': Perceptron(), 
   'grid':{'random_state':[random_state]},
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0},
  {'est': LinearSVC(), 
   'grid':{
      'random_state':[random_state],
      'max_iter':[10000],
   },
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0},
  {'est': RandomForestClassifier(), 
   'grid':{'random_state':[random_state]},
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0},
  {'est': ExtraTreesClassifier(), 
   'grid':{'random_state':[random_state]},
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0},
  {'est': GradientBoostingClassifier(), 
   'grid':{'random_state':[random_state]},
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0},
  {'est': RidgeClassifier(), 
   'grid':{
      'random_state':[random_state],
      'max_iter':[10000],
   },
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0},
  {'est': PassiveAggressiveClassifier(), 
   'grid':{
      'random_state':[random_state],
      'max_iter':[10000],
      'n_jobs':[-1],
   },
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0},
  {'est': GaussianProcessClassifier(), 
   'grid':{
      'random_state':[random_state],
      'n_jobs':[-1],
   },
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0},
  {'est': AdaBoostClassifier(), 
   'grid':{'random_state':[random_state]},
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0},
  {'est': BaggingClassifier(), 
   'grid':{
      'random_state':[random_state],
      'n_jobs':[-1],
      },
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0},
  {'est': BernoulliNB(), 
   'grid':{ },
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0},
  {'est': LinearDiscriminantAnalysis(), 
   'grid':{ },
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0},
  {'est': NearestCentroid(), 
   'grid':{ 
      'metric': ['euclidean', 'l2'], 
      'shrink_threshold': [None, 1e-1],
   },
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0},
  {'est': QuadraticDiscriminantAnalysis(), 
   'grid':{ },
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0},
  {'est': SVC(), 
   'grid':{
      'random_state':[random_state],
      'max_iter':[10000],
    },
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0},
]
```

In [ ]:
# Combinações de classificador/vetorizador
# Vetorizador TfidfVectorizer
# Somente melhor classificador: NearestCentroid 
estimators = [
  {'est': NearestCentroid(), 
   'grid':{ 
      'metric': ['euclidean', 'l2', 'manhattan'], 
      'shrink_threshold': [None, 1e-1],
   },
   'vec': tfidfVectorizer, 'est_name':'', 'vec_name':'', 'precision':0.0, 'recall':0.0, 'accuracy':0.0, 'f1':0.0},
]

In [ ]:
# Combinações de classificador/vetorizador
# Vetorizador CountVectorizer
estimators2 = []
for estimator in estimators:
  item2 = estimator.copy()
  item2['vec'] = countVectorizer
  estimators2.append(item2)

estimators.extend(estimators2)   

In [ ]:
# Treina/testa classificador/vetorizador
for estimator in estimators:
  estimator['est'], estimator['est_name'], estimator['vec_name'], estimator['f1'] = build(df['texto'],df['classe'],estimator['vec'],estimator['est'],estimator['grid'])


Testando o classificador NearestCentroid - TfidfVectorizer ...
Fitting 5 folds for each of 6 candidates, totalling 30 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    

In [ ]:
# Seleção do melhor classificador/vetorizador
def get_f1(estimator):
  return estimator.get('f1')

estimators.sort(key=get_f1, reverse=True)

for estimator in estimators:
  print(estimator['est_name'],'-',estimator['vec_name'],'- F1:',estimator['f1']*100)

# escolhe melhor classificador/vetorizador
clf = estimators[0]['est']
vectorizer = estimators[0]['vec']
print('\nSelecionado: ',estimators[0]['est_name'],'-',estimators[0]['vec_name'])

NearestCentroid - TfidfVectorizer - F1: 48.672078055347576
NearestCentroid - CountVectorizer - F1: 44.57278593508021

Selecionado:  NearestCentroid - TfidfVectorizer


In [ ]:
# escolhe manualmente classificador/vetorizador
#index = 1
#clf = estimators[index]['est']
#vectorizer = estimators[index]['vec']
#print('\nSelecionado: ',estimators[index]['est_name'],'-',estimators[index]['vec_name'])

## Classificando os registros não rotulados para o desafio

Repita os passos acima testando outras preparações de dados, outros vetorizadores, outros parâmetros e outras técnicas de classificação. 

Quando estiver satisfeito com a performance do seu classificador, deve treiná-lo agora com **todos** os registros pré-rotulados. 

Esse classificador será então utilizado para inferir as classes dos registros não rotulados do desafio, como veremos a seguir.

In [ ]:
# Treina o classificador com toda a base fornecida
X_train = vectorizer.transform(df['texto'].tolist()).toarray()
y_train =  df['classe'].tolist()
clf.fit(X_train, y_train)

NearestCentroid(metric='euclidean', shrink_threshold=0.1)

In [ ]:
# Carrega os dados da base não rotulada
path = path_meu_google_drive + nome_arquivo_sem_rotulos
df_test = pd.read_csv(path, index_col=None, engine='python', sep =',', encoding="utf-8")
print('Total de registros carregados:',len(df_test))

Total de registros carregados: 300


In [ ]:
# Prepara os dados para classificação
df_test['texto'] = df_test['texto'].apply(limpar_texto)
df_test.head()

,id,texto
0,270,leopardo faminto dar mal atacar porco espinho ...
1,523,general dissidente sair ileso atentado suicida...
2,155,rezar mim pedir francisco aniversario papar pa...
3,1294,rottweiler participar trabalho demolicao predi...
4,1349,golpe bilhete premiar levar quatro cadeia pr a...


In [ ]:
# Salva os registros de treino pre-processados
path = path_meu_google_drive + nome_arquivo_sem_rotulos_preprocessado
df_test.to_csv(path, index=False, encoding="utf-8", columns=['id','texto'])

In [ ]:
# Vetoriza os textos que serão classificados
X_test = vectorizer.transform(df_test['texto'].tolist()).toarray()

In [ ]:
# Executa a classificação dos registros não rotulados
y_predicted = clf.predict(X_test)
df_test['classe'] = y_predicted

# Exibe uma amostra dos resultados
df_test.head(10)

,id,texto,classe
0,270,leopardo faminto dar mal atacar porco espinho ...,surpresa
1,523,general dissidente sair ileso atentado suicida...,tristeza
2,155,rezar mim pedir francisco aniversario papar pa...,surpresa
3,1294,rottweiler participar trabalho demolicao predi...,surpresa
4,1349,golpe bilhete premiar levar quatro cadeia pr a...,desgosto
5,236,usp unicamp unesp adir inicio aulas causa nova...,medo
6,1691,washington post registrar perdas us dezenove c...,neutro
7,438,parlamento ucraniano aprovar criacao forca def...,neutro
8,677,explosoes bancos assustar municipios interior ...,medo
9,422,governo china prometer pulso firme contra corr...,neutro


In [ ]:
# Salva os registros classificados
path = path_meu_google_drive + nome_arquivo_rotulado_classificador
df_test.to_csv(path, index=False, encoding="utf-8", columns=['id','classe'])